## Все данные будем извлекать с двух ресурсов: [fanfics.me](fanfics.me) и [ficbook.ru](ficbook.ru)

Какие вообще данные в теории понадобятся?
- название
- ссылка
- описание фанфика
- персонажи
- пейринги
- фандомы
- возрастной рейтинг
- теги отношений (слеш/джен/гет/фем)
- тег размера (мини/миди/макси/драббл)
- размер
- статус
- события (можно получить только для фанфиков с [fanfics.me](fanfics.me))
- жанры
- предупреждения

## Что вообще происходит?

Сейчас мы будем доставать все вышеописанные данные с сайта. Для этого нужно открыть страницу со списком фанфиков и достать оттуда все названия, описания, персонажей и так далее. Чтобы произвести эти действия, нужно использовать специализированные библиотеки.

In [1]:
import requests # библиотека для доступа к исходному коду страниц
from bs4 import BeautifulSoup # библиотека для «вытаскивания» нужных нам данных из исходного кода страницы

Каждая страница с фанфикам по Гарри Поттеру на [fanfics.me](fanfics.me) выглядит следующим образом: 
>https://fanfics.me/find?fandom1=2&page=1

где последнее число отвечает за номер страницы. Всего таких страниц с фанфиками 1031 на момент написания этого кода. Давайте вытащим исходный код каждой страницы в цикле.

In [2]:
URLs = ['https://fanfics.me/find?fandom1=2&page=' + str(page) for page in range(1, 3)]
page_sources = [requests.get(url) for url in URLs]

Исходный код надо превратить в «суп», с которым будем удобно работать для поиска информации на странице. «Суп» будем искать для каждой страницы, исходный код которой мы получили. Тем самым получим список с «супами».

In [3]:
soups = [BeautifulSoup(page_source.content, 'html.parser') for page_source in page_sources]

Вся информация о фанфике содержится в контейнере 
```html
<table class='FicTbl'>
```
Поэтому для каждого «супа» найдём все разделы с информацией о фанфиках, тем самым мы получим «суп» для каждого фанфика.

In [4]:
fanfic_soups = []
for soup in soups:
    fanfic_soups += soup.find_all('table', class_='FicTbl')

## Названия
Если внимательно посмотреть на код страницы, то становится ясно, что интересующее нас название лежит в контейнере 
```html
<div class='FicTable_Title'>
``` 
так что ищем этот контейнер (он всего один для каждого «супа», поэтому используем `find` вместо `find_all`)

In [5]:
title_meta = [soup.find('div', class_='FicTable_Title') for soup in fanfic_soups]
title_meta[0]

<div class="FicTable_Title">
<h4><a href="/fic159023">Великолепное лето</a> <span class="small red">(слэш)</span></h4>
<div class="FicTable_Stat"><span class="Views" style="margin-right:20px;" title="Количество просмотров 2 521">2.5k</span>
<span class="ReadersCount" style="margin-right:20px;" title="Количество читателей 197">197</span><span class="Comments" style="margin-right:20px;" title="Количество комментариев 2">2</span><span class="RecommendsCount" style="margin-right:20px;" title="Количество рекомендаций">0</span><span class="DateUpdate" style="margin-right:20px;" title="Опубликовано 03.02.2021, изменено 11.02.2021">11.02.2021</span></div>
</div>

Проблема в том, что контейнер содержит не только название, но и ещё массу лишней информации, поэтому находим уникальный для названия контейнер -- это контейнер `<a>`, он применяется только к названию.

In [6]:
titles = [soup.find('a').text for soup in title_meta]
titles[0]

'Великолепное лето'

## Ссылка

In [7]:
links = ['fanfics.me' + soup.find('a')['href'] for soup in title_meta]
links[0]

'fanfics.me/fic159023'

## Описания
Описание лежит в контейнере 
```html
<td class='FicTbl_sammary'>
``` 
так что ищем этот контейнер (он всего один для каждого «супа», поэтому используем `find` вместо `find_all`)

In [8]:
summaries = [soup.find('td', class_='FicTbl_sammary').text for soup in fanfic_soups]
summaries[0]

'Гарри проводит лето со Снейпом.[2 часть цикла «Арест Гарри Поттера».]'

## Персонажи

In [9]:
import re

In [10]:
characters = []
for soup in fanfic_soups:
    single = [char.text for char in soup.find_all(href=re.compile('/character.*'))]
    pairings = [pair.text.split('/') for pair in soup.find_all(href=re.compile('/paring.*'))]
    for pairing in pairings:
        single += pairing
    single = list(set(single))
    characters.append(', '.join(single))
characters[0]

'Гарри Поттер, Северус Снейп'

## Пейринги

In [11]:
pairings = []
for soup in fanfic_soups:
    fic_pairings = [pair.text for pair in soup.find_all(href=re.compile('/paring.*'))]
    pairings.append(', '.join(fic_pairings))
pairings[0]

'Гарри Поттер/Северус Снейп'

## Фандомы

In [12]:
fandoms = []
for soup in fanfic_soups:
    fic_fandoms = [fandom.text for fandom in soup.find_all(href=re.compile('\/fandom.*|\/index\.php\?section=fandoms.*'))]
    fandoms.append(', '.join(fic_fandoms))
fandoms[0]

'Гарри Поттер'

## Возрастной рейтинг

In [13]:
ratings = [soup.find('span', class_='FicTable_Rating').text for soup in fanfic_soups]
ratings[0]

'NC-17'

## Теги отношений

In [14]:
relationships = [soup.find('div', class_='FicTable_Title').find('h4').find('span').text.strip('()') for soup in fanfic_soups]
relationships[0]

'слэш'

## Теги размера + размер

In [15]:
sizes_full = [soup.find('span', class_='FicTable_Size').text.split('|') for soup in fanfic_soups]
size_tags = [size_full[0].strip() for size_full in sizes_full]
sizes = [size_full[1].strip()[:-3] for size_full in sizes_full]
(size_tags[0], sizes[0])

('Макси', '96')

## Статус

In [16]:
statuses = [soup.find('span', class_='FicTable_Status').text.replace('\xa0', ' ') for soup in fanfic_soups]
statuses[0]

'В процессе'

## События

In [17]:
events = []
for soup in fanfic_soups:
    fic_events = [event.text for event in soup.find_all(href=re.compile('/find\?keyword=.*'))]
    events.append(', '.join(fic_events))
events[0]

'Летом'

## Жанры

In [18]:
meta_unfiltered = [soup.find('td', class_='FicTbl_meta').text for soup in fanfic_soups]
meta_unfiltered[0]

'\nПереводчик:\xa0Дарт Снейпер\n\nФандом:\xa0Гарри Поттер\n\nПерсонажи:\xa0Гарри Поттер/Северус Снейп\nРейтинг:\xa0NC-17\n\nЖанры:\xa0Драма, Романтика\n                    \nРазмер:\xa0Макси | 96 Кб\n\nСтатус:\xa0В\xa0процессе\nПредупреждения:\xa0AU, ООС\n                    События:\xa0Летом\n2\n'

In [19]:
genres_regex = re.compile('Жанры:\\xa0(.*)\\n +')

In [20]:
genres = []
for meta in meta_unfiltered:
    match = re.search(genres_regex, meta)
    if match:
        genres.append(match.group(1))
    else:
        genres.append('')
genres[0]

'Драма, Романтика'

## Предупреждения

In [21]:
warnings_regex = re.compile('Предупреждения:\\xa0(.*)\\n +')

In [22]:
warnings = []
for meta in meta_unfiltered:
    match = re.search(warnings_regex, meta)
    if match:
        warnings.append(match.group(1))
    else:
        warnings.append('')
warnings[0]

'AU, ООС'

## Делаем pandas датафрейм!

In [23]:
data = {
    'title': titles,
    'summary': summaries,
    'characters': characters,
    'pairings': pairings,
    'fandoms': fandoms,
    'rating': ratings,
    'size_tag': size_tags,
    'size': sizes,
    'statuse': statuses,
    'events': events,
    'genres': genres,
    'warnings': warnings,
    'link': links
}

In [24]:
import pandas as pd
fanfictionDF = pd.DataFrame(data, columns = data.keys())
fanfictionDF

,title,summary,characters,pairings,fandoms,rating,size_tag,size,statuse,events,genres,warnings,link
0,Великолепное лето,Гарри проводит лето со Снейпом.[2 часть цикла ...,"Гарри Поттер, Северус Снейп",Гарри Поттер/Северус Снейп,Гарри Поттер,NC-17,Макси,96,В процессе,Летом,"Драма, Романтика","AU, ООС",fanfics.me/fic159023
1,Прожить чужую жизнь,"Очутиться в другом мире, прожить чужую жизнь д...","Невилл Лонгботтом, Гарри Поттер, Гермиона Грей...",Гарри Поттер/Гермиона Грейнджер,Гарри Поттер,R,Макси,418,В процессе,"Адекватные Дурсли, Дурсли - волшебники, Тайный...","Попаданцы, Юмор, Приключения","AU, ООС, От первого лица (POV)",fanfics.me/fic150677
2,История Лорда: Лёд и Пламя,"История моей жизни продолжается, долбаный очка...","Гарри Поттер, Нимфадора Тонкс, Рон Уизли, Драк...",Гарри Поттер/Нимфадора Тонкс,Гарри Поттер,NC-17,Макси,83,В процессе,"Власти магического мира, Наследие, Не в Англии...","Ангст, Драма, Фэнтези, Даркфик, Триллер","Насилие, AU, ООС, От первого лица (POV), Пытки...",fanfics.me/fic134231
3,Цветы в руках твоих,В своём прошлом Гарри подвергается изнасилован...,"Гарри Поттер, Северус Снейп",Гарри Поттер/Северус Снейп,Гарри Поттер,R,Мини,9,Закончен,ПостХогвартс,"Hurt/comfort, Романтика","ООС, Изнасилование, UST",fanfics.me/fic159380
4,Глупый мальчишка,"Маленькое снарри, где Северус всё решил за дво...","Гарри Поттер, Северус Снейп",Гарри Поттер/Северус Снейп,Гарри Поттер,PG-13,Мини,8,Закончен,ПостХогвартс,"Ангст, Драма, Романтика","ООС, От первого лица (POV), UST",fanfics.me/fic159157
5,Аконит,По улицам магического Лондона бродит таинствен...,"Гарри Поттер, Драко Малфой",Гарри Поттер/Драко Малфой,Гарри Поттер,NC-17,Макси,98,В процессе,"Аврорат, Сокрытие магических способностей","Приключения, Романтика, Hurt/comfort, Драма","Нецензурная лексика, UST",fanfics.me/fic159258
6,Бетти Снейп и Чужая Душа,"Австралия, 2014 год. 13-летняя Бетти становитс...","Новый Женский Персонаж, Северус Снейп, Гермион...",Гермиона Грейнджер/Северус Снейп,Гарри Поттер,R,Макси,454,В процессе,"Дети главных героев, Жизнь среди маглов, ПостХ...","Ангст, Драма, Hurt/comfort","Насилие, AU",fanfics.me/fic134189
7,"Сказки барда Бидля, том 2, неоконченный","Две сказки барда Бидля, не вошедшие в первый т...",Неизвестный Персонаж,,Гарри Поттер,General,Мини,8,Закончен,,"Сказка, Пропущенная сцена",,fanfics.me/fic159368
8,Один из нас,О бездействии и о вере в лучшее. В каждом из н...,"Ремус Люпин, Рабастан Лестрейндж, Нарцисса Мал...",Люциус Малфой/Нарцисса Малфой,Гарри Поттер,R,Макси,334,Закончен,"Азкабан, Темная сторона, Аврорат, Времена Маро...","Пропущенная сцена, Ангст, Общий, Экшен",,fanfics.me/fic81384
9,"Рыжий, рыжий",Ещё буквально вчера всё было нормально. Я знал...,"Артур Уизли, Молли Уизли, Гарри Поттер, Джинни...","Гарри Поттер/Рон Уизли, Артур Уизли/Молли Уизли",Гарри Поттер,R,Миди,93,В процессе,Немагическое AU,"Драма, Романтика, AU","AU, ООС",fanfics.me/fic158242


## Пришло время сделать одну большую функцию для всего этого непотребства

In [25]:
genres_regex = re.compile('Жанры:\\xa0(.*)\\n +')
warnings_regex = re.compile('Предупреждения:\\xa0(.*)\\n +')

def processPage(url):
    page_source = requests.get(url)
    soup = BeautifulSoup(page_source.content, 'html.parser')
    fanfic_soups = soup.find_all('table', class_='FicTbl')
    
    characters = []
    pairings = []
    fandoms = []
    ratings = []
    relationships = []
    size_tags = []
    sizes = []
    statuses = []
    events = []
    genres = []
    warnings = []
    titles = []
    links = []
    summaries = []
    
    for soup in fanfic_soups:
        title_meta = soup.find('div', class_='FicTable_Title')
        titles.append(soup.find('a').text)
        links.append('fanfics.me' + soup.find('a')['href'])
        
        summaries.append(soup.find('td', class_='FicTbl_sammary').text)
        
        single = [char.text for char in soup.find_all(href=re.compile('\/character.*'))]
        fic_pairings = [pair.text.split('/') for pair in soup.find_all(href=re.compile('\/paring.*'))]
        for fic_pairing in fic_pairings:
            single += fic_pairing
        single = list(set(single))
        characters.append(', '.join(single))
            
        fic_pairings = [pair.text for pair in soup.find_all(href=re.compile('\/paring.*'))]
        pairings.append(', '.join(fic_pairings))
        
        fic_fandoms = [fandom.text for fandom in soup.find_all(href=re.compile('\/fandom.*|\/index\.php\?section=fandoms.*'))]
        fandoms.append(', '.join(fic_fandoms))
        
        ratings.append(soup.find('span', class_='FicTable_Rating').text)
        relationships.append(soup.find('div', class_='FicTable_Title').find('h4').find('span').text.strip('()'))
    
        size_full = soup.find('span', class_='FicTable_Size').text.split('|')
        size_tags.append(size_full[0].strip())
        sizes.append(size_full[1].strip()[:-3])
    
        statuses.append(soup.find('span', class_='FicTable_Status').text.replace('\xa0', ' '))
        
        fic_events = [event.text for event in soup.find_all(href=re.compile('\/find\?keyword=.*'))]
        events.append(', '.join(fic_events))

        meta_unfiltered = soup.find('td', class_='FicTbl_meta').text
        
        genres_match = re.search(genres_regex, meta_unfiltered)
        if genres_match:
            genres.append(genres_match.group(1))
        else:
            genres.append('')
        
        warnings_match = re.search(warnings_regex, meta_unfiltered)
        if warnings_match:
            warnings.append(warnings_match.group(1))
        else:
            warnings.append('')
            
    data = {
        'title': titles,
        'summary': summaries,
        'characters': characters,
        'pairings': pairings,
        'fandoms': fandoms,
        'rating': ratings,
        'size_tag': size_tags,
        'size': sizes,
        'statuse': statuses,
        'events': events,
        'genres': genres,
        'warnings': warnings,
        'link': links
    }
    
    fanfictionDF = pd.DataFrame(data, columns = data.keys())
    return fanfictionDF

In [26]:
testDF = processPage('https://fanfics.me/find?fandom1=2&page=1')

In [27]:
testDF.to_csv('testDF.csv')
pd.read_csv('testDF.csv', index_col = 0)

,title,summary,characters,pairings,fandoms,rating,size_tag,size,statuse,events,genres,warnings,link
0,Великолепное лето,Гарри проводит лето со Снейпом.[2 часть цикла ...,"Гарри Поттер, Северус Снейп",Гарри Поттер/Северус Снейп,Гарри Поттер,NC-17,Макси,96,В процессе,Летом,"Драма, Романтика","AU, ООС",fanfics.me/fic159023
1,Прожить чужую жизнь,"Очутиться в другом мире, прожить чужую жизнь д...","Невилл Лонгботтом, Гарри Поттер, Гермиона Грей...",Гарри Поттер/Гермиона Грейнджер,Гарри Поттер,R,Макси,418,В процессе,"Адекватные Дурсли, Дурсли - волшебники, Тайный...","Попаданцы, Юмор, Приключения","AU, ООС, От первого лица (POV)",fanfics.me/fic150677
2,История Лорда: Лёд и Пламя,"История моей жизни продолжается, долбаный очка...","Гарри Поттер, Нимфадора Тонкс, Рон Уизли, Драк...",Гарри Поттер/Нимфадора Тонкс,Гарри Поттер,NC-17,Макси,83,В процессе,"Власти магического мира, Наследие, Не в Англии...","Ангст, Драма, Фэнтези, Даркфик, Триллер","Насилие, AU, ООС, От первого лица (POV), Пытки...",fanfics.me/fic134231
3,Цветы в руках твоих,В своём прошлом Гарри подвергается изнасилован...,"Гарри Поттер, Северус Снейп",Гарри Поттер/Северус Снейп,Гарри Поттер,R,Мини,9,Закончен,ПостХогвартс,"Hurt/comfort, Романтика","ООС, Изнасилование, UST",fanfics.me/fic159380
4,Глупый мальчишка,"Маленькое снарри, где Северус всё решил за дво...","Гарри Поттер, Северус Снейп",Гарри Поттер/Северус Снейп,Гарри Поттер,PG-13,Мини,8,Закончен,ПостХогвартс,"Ангст, Драма, Романтика","ООС, От первого лица (POV), UST",fanfics.me/fic159157
5,Аконит,По улицам магического Лондона бродит таинствен...,"Гарри Поттер, Драко Малфой",Гарри Поттер/Драко Малфой,Гарри Поттер,NC-17,Макси,98,В процессе,"Аврорат, Сокрытие магических способностей","Приключения, Романтика, Hurt/comfort, Драма","Нецензурная лексика, UST",fanfics.me/fic159258
6,Бетти Снейп и Чужая Душа,"Австралия, 2014 год. 13-летняя Бетти становитс...","Новый Женский Персонаж, Северус Снейп, Гермион...",Гермиона Грейнджер/Северус Снейп,Гарри Поттер,R,Макси,454,В процессе,"Дети главных героев, Жизнь среди маглов, ПостХ...","Ангст, Драма, Hurt/comfort","Насилие, AU",fanfics.me/fic134189
7,"Сказки барда Бидля, том 2, неоконченный","Две сказки барда Бидля, не вошедшие в первый т...",Неизвестный Персонаж,NaN,Гарри Поттер,General,Мини,8,Закончен,NaN,"Сказка, Пропущенная сцена",NaN,fanfics.me/fic159368
8,Один из нас,О бездействии и о вере в лучшее. В каждом из н...,"Ремус Люпин, Рабастан Лестрейндж, Нарцисса Мал...",Люциус Малфой/Нарцисса Малфой,Гарри Поттер,R,Макси,334,Закончен,"Азкабан, Темная сторона, Аврорат, Времена Маро...","Пропущенная сцена, Ангст, Общий, Экшен",NaN,fanfics.me/fic81384
9,"Рыжий, рыжий",Ещё буквально вчера всё было нормально. Я знал...,"Артур Уизли, Молли Уизли, Гарри Поттер, Джинни...","Гарри Поттер/Рон Уизли, Артур Уизли/Молли Уизли",Гарри Поттер,R,Миди,93,В процессе,Немагическое AU,"Драма, Романтика, AU","AU, ООС",fanfics.me/fic158242


## Теперь надо медленно, но верно проходить по каждой странице и добавлять в датафрейм результаты парсинга

In [28]:
from datetime import datetime

In [31]:
page_link = 'https://fanfics.me/find?fandom1=2&page='
for page_number in range(1, 1037):
    pageDF = processPage(page_link + str(page_number))
    pageDF.to_csv('page_' + str(page_number) + '.csv')
    now = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    print(str(page_number) + ' pages processed at '+ now)

1 pages processed at 11/02/2021 20:25:05
2 pages processed at 11/02/2021 20:25:07
3 pages processed at 11/02/2021 20:25:09
4 pages processed at 11/02/2021 20:25:10
5 pages processed at 11/02/2021 20:25:11
6 pages processed at 11/02/2021 20:25:13
7 pages processed at 11/02/2021 20:25:15
8 pages processed at 11/02/2021 20:25:17
9 pages processed at 11/02/2021 20:25:18
10 pages processed at 11/02/2021 20:25:20
11 pages processed at 11/02/2021 20:25:21
12 pages processed at 11/02/2021 20:25:22
13 pages processed at 11/02/2021 20:25:24
14 pages processed at 11/02/2021 20:25:26
15 pages processed at 11/02/2021 20:25:28
16 pages processed at 11/02/2021 20:25:29
17 pages processed at 11/02/2021 20:25:31
18 pages processed at 11/02/2021 20:25:33
19 pages processed at 11/02/2021 20:25:35
20 pages processed at 11/02/2021 20:25:37
21 pages processed at 11/02/2021 20:25:39
22 pages processed at 11/02/2021 20:25:41
23 pages processed at 11/02/2021 20:25:42
24 pages processed at 11/02/2021 20:25:44
2

195 pages processed at 11/02/2021 20:30:07
196 pages processed at 11/02/2021 20:30:08
197 pages processed at 11/02/2021 20:30:10
198 pages processed at 11/02/2021 20:30:11
199 pages processed at 11/02/2021 20:30:13
200 pages processed at 11/02/2021 20:30:14
201 pages processed at 11/02/2021 20:30:15
202 pages processed at 11/02/2021 20:30:17
203 pages processed at 11/02/2021 20:30:18
204 pages processed at 11/02/2021 20:30:19
205 pages processed at 11/02/2021 20:30:21
206 pages processed at 11/02/2021 20:30:22
207 pages processed at 11/02/2021 20:30:24
208 pages processed at 11/02/2021 20:30:25
209 pages processed at 11/02/2021 20:30:27
210 pages processed at 11/02/2021 20:30:29
211 pages processed at 11/02/2021 20:30:30
212 pages processed at 11/02/2021 20:30:33
213 pages processed at 11/02/2021 20:30:35
214 pages processed at 11/02/2021 20:30:36
215 pages processed at 11/02/2021 20:30:38
216 pages processed at 11/02/2021 20:30:39
217 pages processed at 11/02/2021 20:30:40
218 pages p

386 pages processed at 11/02/2021 20:34:59
387 pages processed at 11/02/2021 20:35:00
388 pages processed at 11/02/2021 20:35:01
389 pages processed at 11/02/2021 20:35:04
390 pages processed at 11/02/2021 20:35:05
391 pages processed at 11/02/2021 20:35:07
392 pages processed at 11/02/2021 20:35:08
393 pages processed at 11/02/2021 20:35:10
394 pages processed at 11/02/2021 20:35:11
395 pages processed at 11/02/2021 20:35:13
396 pages processed at 11/02/2021 20:35:14
397 pages processed at 11/02/2021 20:35:16
398 pages processed at 11/02/2021 20:35:17
399 pages processed at 11/02/2021 20:35:19
400 pages processed at 11/02/2021 20:35:21
401 pages processed at 11/02/2021 20:35:22
402 pages processed at 11/02/2021 20:35:24
403 pages processed at 11/02/2021 20:35:25
404 pages processed at 11/02/2021 20:35:27
405 pages processed at 11/02/2021 20:35:28
406 pages processed at 11/02/2021 20:35:30
407 pages processed at 11/02/2021 20:35:31
408 pages processed at 11/02/2021 20:35:33
409 pages p

577 pages processed at 11/02/2021 20:40:07
578 pages processed at 11/02/2021 20:40:09
579 pages processed at 11/02/2021 20:40:11
580 pages processed at 11/02/2021 20:40:12
581 pages processed at 11/02/2021 20:40:14
582 pages processed at 11/02/2021 20:40:18
583 pages processed at 11/02/2021 20:40:21
584 pages processed at 11/02/2021 20:40:23
585 pages processed at 11/02/2021 20:40:25
586 pages processed at 11/02/2021 20:40:26
587 pages processed at 11/02/2021 20:40:28
588 pages processed at 11/02/2021 20:40:29
589 pages processed at 11/02/2021 20:40:31
590 pages processed at 11/02/2021 20:40:33
591 pages processed at 11/02/2021 20:40:35
592 pages processed at 11/02/2021 20:40:37
593 pages processed at 11/02/2021 20:40:39
594 pages processed at 11/02/2021 20:40:40
595 pages processed at 11/02/2021 20:40:42
596 pages processed at 11/02/2021 20:40:45
597 pages processed at 11/02/2021 20:40:46
598 pages processed at 11/02/2021 20:40:48
599 pages processed at 11/02/2021 20:40:50
600 pages p

768 pages processed at 11/02/2021 20:45:54
769 pages processed at 11/02/2021 20:45:56
770 pages processed at 11/02/2021 20:45:58
771 pages processed at 11/02/2021 20:46:00
772 pages processed at 11/02/2021 20:46:02
773 pages processed at 11/02/2021 20:46:03
774 pages processed at 11/02/2021 20:46:05
775 pages processed at 11/02/2021 20:46:07
776 pages processed at 11/02/2021 20:46:09
777 pages processed at 11/02/2021 20:46:12
778 pages processed at 11/02/2021 20:46:13
779 pages processed at 11/02/2021 20:46:15
780 pages processed at 11/02/2021 20:46:17
781 pages processed at 11/02/2021 20:46:20
782 pages processed at 11/02/2021 20:46:21
783 pages processed at 11/02/2021 20:46:23
784 pages processed at 11/02/2021 20:46:25
785 pages processed at 11/02/2021 20:46:27
786 pages processed at 11/02/2021 20:46:28
787 pages processed at 11/02/2021 20:46:30
788 pages processed at 11/02/2021 20:46:32
789 pages processed at 11/02/2021 20:46:35
790 pages processed at 11/02/2021 20:46:36
791 pages p

959 pages processed at 11/02/2021 20:51:59
960 pages processed at 11/02/2021 20:52:00
961 pages processed at 11/02/2021 20:52:02
962 pages processed at 11/02/2021 20:52:04
963 pages processed at 11/02/2021 20:52:05
964 pages processed at 11/02/2021 20:52:07
965 pages processed at 11/02/2021 20:52:08
966 pages processed at 11/02/2021 20:52:10
967 pages processed at 11/02/2021 20:52:12
968 pages processed at 11/02/2021 20:52:13
969 pages processed at 11/02/2021 20:52:15
970 pages processed at 11/02/2021 20:52:17
971 pages processed at 11/02/2021 20:52:19
972 pages processed at 11/02/2021 20:52:20
973 pages processed at 11/02/2021 20:52:22
974 pages processed at 11/02/2021 20:52:24
975 pages processed at 11/02/2021 20:52:26
976 pages processed at 11/02/2021 20:52:28
977 pages processed at 11/02/2021 20:52:30
978 pages processed at 11/02/2021 20:52:32
979 pages processed at 11/02/2021 20:52:35
980 pages processed at 11/02/2021 20:52:37
981 pages processed at 11/02/2021 20:52:39
982 pages p

## Соединим все сохраненные файлы в один датафрейм, а потом в один файл

In [32]:
fanfictionDF = pd.read_csv('page_1.csv', index_col = 0)
for file_number in range(2, 1037):
    new = pd.read_csv('page_' + str(file_number)+ '.csv', index_col = 0)
    fanfictionDF = pd.concat([fanfictionDF, new], ignore_index = True)
    now = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    print(str(file_number) + ' files processed at '+ now)

fanfictionDF.to_csv('fanfiction.csv')

2 files processed at 11/02/2021 20:55:46
3 files processed at 11/02/2021 20:55:46
4 files processed at 11/02/2021 20:55:46
5 files processed at 11/02/2021 20:55:46
6 files processed at 11/02/2021 20:55:46
7 files processed at 11/02/2021 20:55:46
8 files processed at 11/02/2021 20:55:46
9 files processed at 11/02/2021 20:55:46
10 files processed at 11/02/2021 20:55:46
11 files processed at 11/02/2021 20:55:46
12 files processed at 11/02/2021 20:55:46
13 files processed at 11/02/2021 20:55:46
14 files processed at 11/02/2021 20:55:46
15 files processed at 11/02/2021 20:55:46
16 files processed at 11/02/2021 20:55:46
17 files processed at 11/02/2021 20:55:46
18 files processed at 11/02/2021 20:55:46
19 files processed at 11/02/2021 20:55:46
20 files processed at 11/02/2021 20:55:46
21 files processed at 11/02/2021 20:55:46
22 files processed at 11/02/2021 20:55:46
23 files processed at 11/02/2021 20:55:46
24 files processed at 11/02/2021 20:55:46
25 files processed at 11/02/2021 20:55:46


195 files processed at 11/02/2021 20:55:48
196 files processed at 11/02/2021 20:55:48
197 files processed at 11/02/2021 20:55:49
198 files processed at 11/02/2021 20:55:49
199 files processed at 11/02/2021 20:55:49
200 files processed at 11/02/2021 20:55:49
201 files processed at 11/02/2021 20:55:49
202 files processed at 11/02/2021 20:55:49
203 files processed at 11/02/2021 20:55:49
204 files processed at 11/02/2021 20:55:49
205 files processed at 11/02/2021 20:55:49
206 files processed at 11/02/2021 20:55:49
207 files processed at 11/02/2021 20:55:49
208 files processed at 11/02/2021 20:55:49
209 files processed at 11/02/2021 20:55:49
210 files processed at 11/02/2021 20:55:49
211 files processed at 11/02/2021 20:55:49
212 files processed at 11/02/2021 20:55:49
213 files processed at 11/02/2021 20:55:49
214 files processed at 11/02/2021 20:55:49
215 files processed at 11/02/2021 20:55:49
216 files processed at 11/02/2021 20:55:49
217 files processed at 11/02/2021 20:55:49
218 files p

386 files processed at 11/02/2021 20:55:52
387 files processed at 11/02/2021 20:55:52
388 files processed at 11/02/2021 20:55:52
389 files processed at 11/02/2021 20:55:52
390 files processed at 11/02/2021 20:55:52
391 files processed at 11/02/2021 20:55:52
392 files processed at 11/02/2021 20:55:52
393 files processed at 11/02/2021 20:55:52
394 files processed at 11/02/2021 20:55:52
395 files processed at 11/02/2021 20:55:52
396 files processed at 11/02/2021 20:55:52
397 files processed at 11/02/2021 20:55:52
398 files processed at 11/02/2021 20:55:52
399 files processed at 11/02/2021 20:55:52
400 files processed at 11/02/2021 20:55:52
401 files processed at 11/02/2021 20:55:52
402 files processed at 11/02/2021 20:55:52
403 files processed at 11/02/2021 20:55:52
404 files processed at 11/02/2021 20:55:52
405 files processed at 11/02/2021 20:55:52
406 files processed at 11/02/2021 20:55:53
407 files processed at 11/02/2021 20:55:53
408 files processed at 11/02/2021 20:55:53
409 files p

579 files processed at 11/02/2021 20:55:56
580 files processed at 11/02/2021 20:55:56
581 files processed at 11/02/2021 20:55:56
582 files processed at 11/02/2021 20:55:56
583 files processed at 11/02/2021 20:55:57
584 files processed at 11/02/2021 20:55:57
585 files processed at 11/02/2021 20:55:57
586 files processed at 11/02/2021 20:55:57
587 files processed at 11/02/2021 20:55:57
588 files processed at 11/02/2021 20:55:57
589 files processed at 11/02/2021 20:55:57
590 files processed at 11/02/2021 20:55:57
591 files processed at 11/02/2021 20:55:57
592 files processed at 11/02/2021 20:55:57
593 files processed at 11/02/2021 20:55:57
594 files processed at 11/02/2021 20:55:57
595 files processed at 11/02/2021 20:55:57
596 files processed at 11/02/2021 20:55:57
597 files processed at 11/02/2021 20:55:57
598 files processed at 11/02/2021 20:55:57
599 files processed at 11/02/2021 20:55:57
600 files processed at 11/02/2021 20:55:57
601 files processed at 11/02/2021 20:55:57
602 files p

774 files processed at 11/02/2021 20:56:02
775 files processed at 11/02/2021 20:56:02
776 files processed at 11/02/2021 20:56:02
777 files processed at 11/02/2021 20:56:02
778 files processed at 11/02/2021 20:56:02
779 files processed at 11/02/2021 20:56:02
780 files processed at 11/02/2021 20:56:02
781 files processed at 11/02/2021 20:56:02
782 files processed at 11/02/2021 20:56:02
783 files processed at 11/02/2021 20:56:02
784 files processed at 11/02/2021 20:56:02
785 files processed at 11/02/2021 20:56:02
786 files processed at 11/02/2021 20:56:02
787 files processed at 11/02/2021 20:56:02
788 files processed at 11/02/2021 20:56:02
789 files processed at 11/02/2021 20:56:02
790 files processed at 11/02/2021 20:56:02
791 files processed at 11/02/2021 20:56:02
792 files processed at 11/02/2021 20:56:02
793 files processed at 11/02/2021 20:56:02
794 files processed at 11/02/2021 20:56:02
795 files processed at 11/02/2021 20:56:02
796 files processed at 11/02/2021 20:56:02
797 files p

970 files processed at 11/02/2021 20:56:08
971 files processed at 11/02/2021 20:56:08
972 files processed at 11/02/2021 20:56:08
973 files processed at 11/02/2021 20:56:08
974 files processed at 11/02/2021 20:56:08
975 files processed at 11/02/2021 20:56:08
976 files processed at 11/02/2021 20:56:08
977 files processed at 11/02/2021 20:56:08
978 files processed at 11/02/2021 20:56:08
979 files processed at 11/02/2021 20:56:08
980 files processed at 11/02/2021 20:56:08
981 files processed at 11/02/2021 20:56:08
982 files processed at 11/02/2021 20:56:08
983 files processed at 11/02/2021 20:56:08
984 files processed at 11/02/2021 20:56:09
985 files processed at 11/02/2021 20:56:09
986 files processed at 11/02/2021 20:56:09
987 files processed at 11/02/2021 20:56:09
988 files processed at 11/02/2021 20:56:09
989 files processed at 11/02/2021 20:56:09
990 files processed at 11/02/2021 20:56:09
991 files processed at 11/02/2021 20:56:09
992 files processed at 11/02/2021 20:56:09
993 files p

In [33]:
fanfictionDF

,title,summary,characters,pairings,fandoms,rating,size_tag,size,statuse,events,genres,warnings,link
0,Великолепное лето,Гарри проводит лето со Снейпом.[2 часть цикла ...,"Гарри Поттер, Северус Снейп",Гарри Поттер/Северус Снейп,Гарри Поттер,NC-17,Макси,96,В процессе,Летом,"Драма, Романтика","AU, ООС",fanfics.me/fic159023
1,Прожить чужую жизнь,"Очутиться в другом мире, прожить чужую жизнь д...","Невилл Лонгботтом, Гарри Поттер, Гермиона Грей...",Гарри Поттер/Гермиона Грейнджер,Гарри Поттер,R,Макси,418,В процессе,"Адекватные Дурсли, Дурсли - волшебники, Тайный...","Попаданцы, Юмор, Приключения","AU, ООС, От первого лица (POV)",fanfics.me/fic150677
2,История Лорда: Лёд и Пламя,"История моей жизни продолжается, долбаный очка...","Гарри Поттер, Нимфадора Тонкс, Рон Уизли, Драк...",Гарри Поттер/Нимфадора Тонкс,Гарри Поттер,NC-17,Макси,83,В процессе,"Власти магического мира, Наследие, Не в Англии...","Ангст, Драма, Фэнтези, Даркфик, Триллер","Насилие, AU, ООС, От первого лица (POV), Пытки...",fanfics.me/fic134231
3,Цветы в руках твоих,В своём прошлом Гарри подвергается изнасилован...,"Гарри Поттер, Северус Снейп",Гарри Поттер/Северус Снейп,Гарри Поттер,R,Мини,9,Закончен,ПостХогвартс,"Hurt/comfort, Романтика","ООС, Изнасилование, UST",fanfics.me/fic159380
4,Глупый мальчишка,"Маленькое снарри, где Северус всё решил за дво...","Гарри Поттер, Северус Снейп",Гарри Поттер/Северус Снейп,Гарри Поттер,PG-13,Мини,8,Закончен,ПостХогвартс,"Ангст, Драма, Романтика","ООС, От первого лица (POV), UST",fanfics.me/fic159157
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20708,Наблюдая Звёзды,"Отношения Гарри и Снейпа становятся лучше, но ...","Гарри Поттер, Северус Снейп",Гарри Поттер/Северус Снейп,Гарри Поттер,NC-17,Макси,18,Заморожен,NaN,"Драма, Романтика, Первый раз",NaN,fanfics.me/fic37
20709,Time and time again,Гарри завтра женится. А сегодня ночью Драко на...,"Джинни Уизли, Гарри Поттер, Драко Малфой","Гарри Поттер/Драко Малфой, Гарри Поттер/Джинни...",Гарри Поттер,R,Мини,12,Закончен,NaN,"Ангст, Романтика",NaN,fanfics.me/fic38
20710,Больше чем друзья,Дружба между юношей и девушкой рано или поздно...,"Гарри Поттер, Гермиона Грейнджер",Гарри Поттер/Гермиона Грейнджер,Гарри Поттер,NC-17,Мини,16,Закончен,NaN,Романтика,NaN,fanfics.me/fic39
20711,"Люби меня, люблю тебя","Реализация замыслов хороша, когда она случайна.","Гарри Поттер, Гермиона Грейнджер",Гарри Поттер/Гермиона Грейнджер,Гарри Поттер,NC-17,Мини,19,Закончен,NaN,"Романтика, Юмор",NaN,fanfics.me/fic40


In [34]:
fanfictionDF = pd.read_csv('fanfiction.csv', index_col = 0)
fanfictionDF

,title,summary,characters,pairings,fandoms,rating,size_tag,size,statuse,events,genres,warnings,link
0,Великолепное лето,Гарри проводит лето со Снейпом.[2 часть цикла ...,"Гарри Поттер, Северус Снейп",Гарри Поттер/Северус Снейп,Гарри Поттер,NC-17,Макси,96,В процессе,Летом,"Драма, Романтика","AU, ООС",fanfics.me/fic159023
1,Прожить чужую жизнь,"Очутиться в другом мире, прожить чужую жизнь д...","Невилл Лонгботтом, Гарри Поттер, Гермиона Грей...",Гарри Поттер/Гермиона Грейнджер,Гарри Поттер,R,Макси,418,В процессе,"Адекватные Дурсли, Дурсли - волшебники, Тайный...","Попаданцы, Юмор, Приключения","AU, ООС, От первого лица (POV)",fanfics.me/fic150677
2,История Лорда: Лёд и Пламя,"История моей жизни продолжается, долбаный очка...","Гарри Поттер, Нимфадора Тонкс, Рон Уизли, Драк...",Гарри Поттер/Нимфадора Тонкс,Гарри Поттер,NC-17,Макси,83,В процессе,"Власти магического мира, Наследие, Не в Англии...","Ангст, Драма, Фэнтези, Даркфик, Триллер","Насилие, AU, ООС, От первого лица (POV), Пытки...",fanfics.me/fic134231
3,Цветы в руках твоих,В своём прошлом Гарри подвергается изнасилован...,"Гарри Поттер, Северус Снейп",Гарри Поттер/Северус Снейп,Гарри Поттер,R,Мини,9,Закончен,ПостХогвартс,"Hurt/comfort, Романтика","ООС, Изнасилование, UST",fanfics.me/fic159380
4,Глупый мальчишка,"Маленькое снарри, где Северус всё решил за дво...","Гарри Поттер, Северус Снейп",Гарри Поттер/Северус Снейп,Гарри Поттер,PG-13,Мини,8,Закончен,ПостХогвартс,"Ангст, Драма, Романтика","ООС, От первого лица (POV), UST",fanfics.me/fic159157
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20708,Наблюдая Звёзды,"Отношения Гарри и Снейпа становятся лучше, но ...","Гарри Поттер, Северус Снейп",Гарри Поттер/Северус Снейп,Гарри Поттер,NC-17,Макси,18,Заморожен,NaN,"Драма, Романтика, Первый раз",NaN,fanfics.me/fic37
20709,Time and time again,Гарри завтра женится. А сегодня ночью Драко на...,"Джинни Уизли, Гарри Поттер, Драко Малфой","Гарри Поттер/Драко Малфой, Гарри Поттер/Джинни...",Гарри Поттер,R,Мини,12,Закончен,NaN,"Ангст, Романтика",NaN,fanfics.me/fic38
20710,Больше чем друзья,Дружба между юношей и девушкой рано или поздно...,"Гарри Поттер, Гермиона Грейнджер",Гарри Поттер/Гермиона Грейнджер,Гарри Поттер,NC-17,Мини,16,Закончен,NaN,Романтика,NaN,fanfics.me/fic39
20711,"Люби меня, люблю тебя","Реализация замыслов хороша, когда она случайна.","Гарри Поттер, Гермиона Грейнджер",Гарри Поттер/Гермиона Грейнджер,Гарри Поттер,NC-17,Мини,19,Закончен,NaN,"Романтика, Юмор",NaN,fanfics.me/fic40
